# MLP Regression for 2(a) and 2(b)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Define sigmoid and its derivative
def sigmoid(h):
    h = np.clip(h, -50, 50)
    return 1.0 / (1.0 + np.exp(-h))

def sigmoid_prime(h):
    s = sigmoid(h)
    return s * (1.0 - s)

# Define the MLP class (one hidden layer, sigmoid activation)
class OneHiddenMLP:
    def __init__(self, n_inputs, n_hidden, alpha0=0.5, eta=0.9, eps=1e-3, max_epochs=2000, rng=None):
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.alpha0 = alpha0
        self.eta = eta
        self.eps = eps
        self.max_epochs = max_epochs
        self.rng = rng if rng is not None else np.random.default_rng()

        # We include bias as an extra "input" with fixed value -1
        self.W0 = self.rng.uniform(-0.1, 0.1, size=(n_hidden, n_inputs + 1))  # Hidden layer weights
        self.W1 = self.rng.uniform(-0.1, 0.1, size=(n_hidden + 1,))           # Output layer weights

    def _forward(self, X):
        N = X.shape[0]
        Xb = np.hstack([ -np.ones((N, 1)), X ])   # Add bias to inputs
        h = Xb @ self.W0.T                          # Hidden layer
        H = sigmoid(h)                              # Sigmoid activations

        Hb = np.hstack([ -np.ones((N, 1)), H ])     # Add bias to hidden layer
        o_in = Hb @ self.W1                         # Output layer
        O = o_in                                    # Linear output (g(h) = h)
        return Xb, h, H, Hb, o_in, O

    def predict(self, X):
        _, _, _, _, _, O = self._forward(X)
        return O

    def fit(self, X, y):
        N = X.shape[0]
        alpha = self.alpha0
        epoch = 0

        while alpha > self.eps and epoch < self.max_epochs:
            Xb, h, H, Hb, o_in, O = self._forward(X)

            e = y - O                             # Errors
            delta_out = (O - y) / N               # Derivative for output
            grad_W1 = Hb.T @ delta_out            # Gradient for W1

            tmp = delta_out[:, None] * self.W1[1:][None, :]
            delta_hidden = sigmoid_prime(h) * tmp  # Gradient for hidden layer
            grad_W0 = delta_hidden.T @ Xb         # Gradient for W0

            # Update weights
            self.W1 -= alpha * grad_W1
            self.W0 -= alpha * grad_W0

            # Decay learning rate
            alpha *= self.eta
            epoch += 1
        return

# Training function that selects the best number of hidden units (J)
def train_mlp_and_select_hidden(X_train, y_train, X_test, y_test,
                                n_inputs,
                                J0=3, alpha0=0.5, eta=0.9, eps=1e-3,
                                max_hidden=20):
    rng = np.random.default_rng(123)

    J = J0
    best_J = J
    best_test_mse = np.inf
    best_model = None
    results = []

    while J <= max_hidden:
        print(f"
Training MLP with J = {J} hidden units...")
        model = OneHiddenMLP(n_inputs, J, alpha0=alpha0, eta=eta, eps=eps, max_epochs=2000, rng=rng)
        model.fit(X_train, y_train)

        y_train_hat_s = model.predict(X_train)
        y_test_hat_s  = model.predict(X_test)

        train_sse = np.sum((y_train - y_train_hat_s) ** 2)
        test_sse  = np.sum((y_test  - y_test_hat_s) ** 2)
        test_mse  = test_sse / len(y_test)
        test_var  = np.var((y_test - y_test_hat_s) ** 2, ddof=1)

        print(f"J = {J}: train SSE = {train_sse:.4f}, test MSE = {test_mse:.4f}")

        results.append((J, train_sse, test_mse, test_var, model))

        if test_mse < best_test_mse:
            best_test_mse = test_mse
            best_J = J
            best_model = model
            J += 1  # try more units
        else:
            # Test error did not improve -> stop
            break

    print(f"
Best number of hidden units (approx): J = {best_J}")
    return best_J, best_model, results

# Now run the function with our dataset
best_J, best_model, results = train_mlp_and_select_hidden(
    X_train, y_train_s, X_test, y_test_s,
    n_inputs=1,  # One input: x
    J0=3, alpha0=0.5, eta=0.9, eps=1e-3, max_hidden=15
)

# Plotting the final result for 2(a)
xs = np.linspace(x_all.min(), x_all.max(), 200)
zs = (xs - x_mean) / x_std  # Standardize the input values
Xs = zs.reshape(-1, 1)
ys_hat_s = best_model.predict(Xs)
ys_hat = y_mean + y_std * ys_hat_s

# Plot the results
plt.figure()
plt.scatter(x_train, y_train, label="Train", alpha=0.6)
plt.scatter(x_test,  y_test,  label="Test",  alpha=0.6)
plt.plot(xs, ys_hat, label=f"MLP 2(a), J={best_J}", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.title("2(a) MLP regression (input = standardized x)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Now run the function with our dataset for 2(b) MLP model (x and x^2 as input)
best_J, best_model, results = train_mlp_and_select_hidden(
    X_train, y_train_s, X_test, y_test_s,
    n_inputs=2,  # Two inputs: x and x^2
    J0=3, alpha0=0.5, eta=0.9, eps=1e-3, max_hidden=15
)

# Output the results for 2(b)
best_J, results[-1]  # The best number of hidden units and the last result (best performance)

# Plotting the final result for 2(b)
xs = np.linspace(x_all.min(), x_all.max(), 200)
zs = (xs - x_mean) / x_std  # Standardize the input values
Xs = np.column_stack([zs, zs**2])
ys_hat_s = best_model.predict(Xs)
ys_hat = y_mean + y_std * ys_hat_s

# Plot the results for 2(b)
plt.figure()
plt.scatter(x_train, y_train, label="Train", alpha=0.6)
plt.scatter(x_test,  y_test,  label="Test",  alpha=0.6)
plt.plot(xs, ys_hat, label=f"MLP 2(b), J={best_J}", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.title("2(b) MLP regression (inputs = standardized x, x^2)")
plt.legend()
plt.grid(True)
plt.show()
